## Code Reproducing Warning!
Everything below is done on Ubuntu 20.04  
First of all, to install and import implicit library I had to create new python virtual environment:  
`python -m venv ./python3.8_rec_sys`  
Then I had to install implicit==0.4.4  
Also I've executed  
`python -m ipykernel install --user --name=my-virtualenv-name`  
to be able to choose the needed kernel in jupyter notebook  
  
If you do something different, you may face some problems, like 'kernel death'

In [61]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix, coo_matrix

# Детерминированные алгоритмы
from implicit.nearest_neighbours import ItemItemRecommender, CosineRecommender, TFIDFRecommender, BM25Recommender

# Метрики
from implicit.evaluation import train_test_split
from implicit.evaluation import precision_at_k, mean_average_precision_at_k, AUC_at_k, ndcg_at_k

In [62]:
data = pd.read_csv('./data/retail_train.csv')
data.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [63]:
test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

# Задание 1. Weighted Random Recommendation

Напишите код для случайных рекоммендаций, в которых вероятность рекомендовать товар прямо пропорциональна логарифму продаж
- Можно сэмплировать товары случайно, но пропорционально какому-либо весу
- Например, прямопропорционально популярности. Вес = log(sales_sum товара)

Функция `np.random.choice` принимает аргумент `p`, в который можно передать список коэффициентов, который учтется при рандомном выборе чисел  
  
1. Создадим датафрейм `items_data`, сгруппировав данные по `item_id` и сумме `sales_value`   
2. Так как мы будем брать логарифм от колонки `sales_value`, заранее заменим все значения < 1 в этой колонке на 1, так как логарифм единицы и так будет 0, а отрицательные коэффициенты в рандом функцию нам не подходят  
3. Добавляем к датафрейму колонку `weights`, которая является натуральным логарифмом колонки `sales_value`  
4. Скейлим всю колонку до суммы в единицу, просто разделив каждое значение на сумму колонки  
5. Проверяем сумму колонки  

In [64]:
items_data = data.groupby('item_id')['sales_value'].sum().reset_index()
items_data.loc[items_data['sales_value'] < 1, ('sales_value')] = 1
items_data['weights'] = np.log(items_data['sales_value'])
items_data['weights'] = items_data['weights'] / items_data['weights'].sum()
items_data['weights'].sum()

0.9999999999999999

In [65]:
def weighted_random_recommendation(items, weights, n=5):
    """Случайные рекоммендации
    
    Input
    -----
    items_weights: pd.DataFrame
        Датафрейм со столбцами item_id, weight. Сумма weight по всем товарам = 1
    """
    
    recs = np.random.choice(items, size=n, p=weights, replace=False)
    
    return recs.tolist()

Проверяем рекомендации

In [66]:
items = items_data.item_id
weights = items_data.weights

weighted_random_recommendation(items, weights)

[6632283, 1056439, 10149851, 1045966, 924262]

# Задание 2. Расчет метрик
Рассчитайте Precision@5 для каждого алгоритма с помощью функции из вебинара 1. Какой алгоритм показывает лучшее качество?

Я сохранил данные из ноутбука на вебинаре в пикл формате, так как .csv превратил списки в строки, а это требовало бы лишней обработки  
Теперь загружаю сохраненный пикл

In [67]:
result = pd.read_pickle('./data/results.pkl')
result.head(2)

,user_id,actual,random_recommendation,popular_recommendation,itemitem,cosine,tfidf,own_purchases
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[5550783, 1077419, 1536833, 13213651, 5592484]","[6534178, 6533889, 1029743, 6534166, 1082185]","[6666, 1082185, 981760, 1127831, 995242]","[1082185, 6666, 981760, 1127831, 1098066]","[1082185, 981760, 1127831, 6666, 1098066]","[1082185, 995242, 1029743, 840361, 904360]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[1007805, 1204210, 852178, 864139, 6396584]","[6534178, 6533889, 1029743, 6534166, 1082185]","[6666, 1082185, 981760, 1098066, 995242]","[1082185, 1098066, 981760, 6666, 826249]","[1082185, 981760, 1098066, 826249, 6666]","[1082185, 1098066, 6534178, 826249, 1127831]"


In [68]:
def precision_at_k(recommended_list, bought_list, k=5):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list[:k])
    
    flags = np.isin(bought_list, recommended_list)
    
    return flags.sum() / len(recommended_list)

In [69]:
for name_col in result.columns[1:]:
    print(f'precision@K = {result.apply(lambda row: precision_at_k(row[name_col], row.actual), axis=1).mean():.3f} for {name_col}')

precision@K = 1.000 for actual
precision@K = 0.001 for random_recommendation
precision@K = 0.155 for popular_recommendation
precision@K = 0.137 for itemitem
precision@K = 0.133 for cosine
precision@K = 0.139 for tfidf
precision@K = 0.220 for own_purchases


/tmp/ipykernel_3929/1849024894.py:8: RuntimeWarning: invalid value encountered in long_scalars
  return flags.sum() / len(recommended_list)


# Задание 3*. Улучшение бейзлайнов и ItemItem

- Попробуйте улучшить бейзлайны, считая их на топ-5000 товаров
- Попробуйте улучшить разные варианты ItemItemRecommender, выбирая число соседей $K$.

Сделаем еще один датафрейм для новых результатов

In [70]:
result_own = data_test.groupby('user_id')['item_id'].unique().reset_index()
result_own.columns=['user_id', 'actual']
result_own.head(2)

,user_id,actual
0,1,"[821867, 834484, 856942, 865456, 889248, 90795..."
1,3,"[835476, 851057, 872021, 878302, 879948, 90963..."


## Random Recommendations (top-500)

Сделаем рандомную рекомендацию, но рандомные товары она будет предсказывать только из 5000 лучших, а не из всего многообразия

In [71]:
items_data.sort_values('sales_value', ascending=False, inplace=True)
items_data.head(2)

,item_id,sales_value,weights
56233,6534178,467993.62,0.000056
56193,6533889,42645.75,0.000046


In [72]:
items_data['weights5000'] = np.log(items_data['sales_value'])
items_data = items_data.reset_index(drop=True)
items_data.loc[5000:, 'weights5000'] = 0
items_data['weights5000'] = items_data['weights5000'] / items_data['weights5000'].sum()
items_data['weights5000'].sum()

0.9999999999999999

In [73]:
def random_recommendation(items, n=5):
    """Случайные рекоммендации"""
    
    items = np.array(items)
    recs = np.random.choice(items, size=n, replace=False)
    
    return recs.tolist()

In [74]:
%%time

# pass only top 5000 valuable products
items = items_data.item_id.head(5000)

result_own['random_recommendation'] = result_own['user_id'].apply(lambda x: random_recommendation(items, n=5))

result_own.head(2)

CPU times: user 272 ms, sys: 3.34 ms, total: 275 ms
Wall time: 274 ms


,user_id,actual,random_recommendation
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[846811, 1075313, 848783, 9296977, 999625]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[1135868, 839818, 838563, 1030093, 1012873]"


## Weighted Random Recommendations (top-500)

In [75]:
%%time

# pass only top 5000 valuable products
items = items_data.item_id.head(5000)
weights = items_data.weights5000.head(5000)

result_own['weighted_random_recommendation'] = result_own['user_id'].apply(lambda x: weighted_random_recommendation(items, weights))

result_own.head(2)

CPU times: user 405 ms, sys: 0 ns, total: 405 ms
Wall time: 403 ms


,user_id,actual,random_recommendation,weighted_random_recommendation
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[846811, 1075313, 848783, 9296977, 999625]","[843558, 915349, 861272, 911974, 5568489]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[1135868, 839818, 838563, 1030093, 1012873]","[1067695, 999581, 1017195, 1134296, 884186]"


### Сравнение с предыдущими бейзлайнами:
precision@K = 0.001 for random_recommendation  
precision@K = 0.155 for popular_recommendation  

In [76]:
for name_col in result_own.columns[1:]:
    print(f'precision@K = {result_own.apply(lambda row: precision_at_k(row[name_col], row.actual), axis=1).mean():.3f} for {name_col}')

precision@K = 1.000 for actual
precision@K = 0.005 for random_recommendation
precision@K = 0.006 for weighted_random_recommendation


Рандомные предсказания стали в 5 раз лучше, а взвешенные рандомные - в 7 раз. Конечно это далеко от нормальных алгоритмов, но тем не менее, польза в методе выборки только лучших товаров - подтверждена

# Item-Item Recommender

Подготовка данных для ItemItem Recommender'a

In [77]:
popularity = data_train.groupby('item_id')['quantity'].sum().reset_index()
popularity.rename(columns={'quantity': 'n_sold'}, inplace=True)

popularity.head()

,item_id,n_sold
0,25671,6
1,26081,1
2,26093,1
3,26190,1
4,26355,2


In [78]:
top_5000 = popularity.sort_values('n_sold', ascending=False).head(5000).item_id.tolist()

In [79]:
# Заведем фиктивный item_id (если юзер покупал товары из топ-5000, то он "купил" такой товар)
data_train.loc[ ~ data_train['item_id'].isin(top_5000), 'item_id'] = 6666

In [80]:
user_item_matrix = pd.pivot_table(data_train, 
                                  index='user_id', columns='item_id', 
                                  values='quantity',
                                  aggfunc='count', 
                                  fill_value=0
                                 )

user_item_matrix[user_item_matrix > 0] = 1 # так как в итоге хотим предсказать 

user_item_matrix = user_item_matrix.astype(float) # необходимый тип матрицы для implicit

# переведем в формат sparse matrix
sparse_user_item = csr_matrix(user_item_matrix).tocsr()

In [81]:
# создаем словари мапинга между id бизнеса к строчному id матрицы

userids = user_item_matrix.index.values
itemids = user_item_matrix.columns.values

matrix_userids = np.arange(len(userids))
matrix_itemids = np.arange(len(itemids))

id_to_itemid = dict(zip(matrix_itemids, itemids))
id_to_userid = dict(zip(matrix_userids, userids))

itemid_to_id = dict(zip(itemids, matrix_itemids))
userid_to_id = dict(zip(userids, matrix_userids))

**Вариант с К=1**

In [82]:
%%time

model = ItemItemRecommender(K=1, num_threads=4) # K - кол-во билжайших соседей

model.fit(csr_matrix(user_item_matrix).T.tocsr(),  # На вход item-user matrix
          show_progress=True)

recs = model.recommend(userid=userid_to_id[2],  # userid - id от 0 до N
                        user_items=csr_matrix(user_item_matrix).tocsr(),   # на вход user-item matrix
                        N=5, # кол-во рекомендаций 
                        filter_already_liked_items=False, 
                        filter_items=None, 
                        recalculate_user=True)

  0%|          | 0/5001 [00:00<?, ?it/s]

CPU times: user 1.93 s, sys: 3.1 ms, total: 1.93 s
Wall time: 797 ms


In [83]:
result_own['itemitem'] = result_own['user_id'].apply(lambda user_id: [
                                     id_to_itemid[rec[0]]  for rec in model.recommend(userid=userid_to_id[user_id], 
                                                user_items=sparse_user_item,   # на вход user-item matrix
                                                N=5, 
                                                filter_already_liked_items=False, 
                                                filter_items=None, 
                                                recalculate_user=True)
                                                             ])

In [84]:
result_own['itemitem1'] = result_own['user_id'].apply(lambda user_id: [
                                     id_to_itemid[rec[0]]  for rec in model.recommend(userid=userid_to_id[user_id], 
                                                user_items=sparse_user_item,   # на вход user-item matrix
                                                N=5, 
                                                filter_already_liked_items=False, 
                                                filter_items=[itemid_to_id[6666]], 
                                                recalculate_user=True)
                                                             ])

**Вариант с К=5 и без товара "6666"**

In [85]:
%%time

model = ItemItemRecommender(K=5, num_threads=4) # K - кол-во билжайших соседей

model.fit(csr_matrix(user_item_matrix).T.tocsr(),  # На вход item-user matrix
          show_progress=True)

recs = model.recommend(userid=userid_to_id[2],  # userid - id от 0 до N
                        user_items=csr_matrix(user_item_matrix).tocsr(),   # на вход user-item matrix
                        N=5, # кол-во рекомендаций 
                        filter_already_liked_items=False, 
                        filter_items=None, 
                        recalculate_user=True)

  0%|          | 0/5001 [00:00<?, ?it/s]

CPU times: user 2.02 s, sys: 7.66 ms, total: 2.03 s
Wall time: 819 ms


In [86]:
result_own['itemitem2'] = result_own['user_id'].apply(lambda user_id: [
                                     id_to_itemid[rec[0]]  for rec in model.recommend(userid=userid_to_id[user_id], 
                                                user_items=sparse_user_item,   # на вход user-item matrix
                                                N=5, 
                                                filter_already_liked_items=False, 
                                                filter_items=None, 
                                                recalculate_user=True)
                                                             ])

In [87]:
result_own['itemitem3'] = result_own['user_id'].apply(lambda user_id: [
                                     id_to_itemid[rec[0]]  for rec in model.recommend(userid=userid_to_id[user_id], 
                                                user_items=sparse_user_item,   # на вход user-item matrix
                                                N=5, 
                                                filter_already_liked_items=False, 
                                                filter_items=[itemid_to_id[6666]], 
                                                recalculate_user=True)
                                                             ])

In [88]:
result_own.head(2)

,user_id,actual,random_recommendation,weighted_random_recommendation,itemitem,itemitem1,itemitem2,itemitem3
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[846811, 1075313, 848783, 9296977, 999625]","[843558, 915349, 861272, 911974, 5568489]","[6666, 1082185, 995242, 1029743, 840361]","[1082185, 995242, 1029743, 840361, 904360]","[6666, 1082185, 981760, 1127831, 995242]","[1082185, 981760, 1127831, 995242, 1098066]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[1135868, 839818, 838563, 1030093, 1012873]","[1067695, 999581, 1017195, 1134296, 884186]","[6666, 1082185, 1098066, 6534178, 826249]","[1082185, 1098066, 6534178, 826249, 1127831]","[6666, 1082185, 981760, 1098066, 995242]","[1082185, 981760, 1098066, 995242, 826249]"


### Сравнение с предыдущим ItemItem Recommender'oм:  
precision@K = 0.137 for itemitem  

In [89]:
for name_col in result_own.columns[4:]:
    print(f'precision@K = {result_own.apply(lambda row: precision_at_k(row[name_col], row.actual), axis=1).mean():.3f} for {name_col}')

precision@K = 0.192 for itemitem
precision@K = 0.220 for itemitem1
precision@K = 0.137 for itemitem2
precision@K = 0.151 for itemitem3


/tmp/ipykernel_3929/1849024894.py:8: RuntimeWarning: invalid value encountered in long_scalars
  return flags.sum() / len(recommended_list)


## ВЫВОДЫ:
На этих данных, средняя точность алгоритма предсказаний оказалась выше у алгоритма с К соседей = 1.  
По разным причинам, такой подход действительно может улучшать метрики качества.  
Кроме того, достаточно сильный буст дает трюк с оставлением только топ 5000 товаров.  
Конкретнее, если их в итоге совсем исключить из рекомендаций, точность алгоритма сразу возрастает на примерно 0.02, что достаточно много.